In [ ]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [ ]:
def readTimeseries():
    timeseries = pd.read_csv(
        'zeitreihe-tagesdaten.csv',
        low_memory = False,
        usecols = ['date', 'bundesland', 'gemeindeschluessel', 'betten_belegt', 'betten_frei'],
        parse_dates = ['date'],
        date_parser = lambda dateStr: pd.to_datetime(dateStr, format = "%Y-%m-%d"),
        dtype = {
            'gemeindeschluessel': 'string',
            'bundesland': 'string'
            })
    return timeseries.sort_values(by = 'date', ascending = True)


In [ ]:
def readKreise():
    kreise = pd.read_excel(
        '04-kreise.xlsx',
        sheet_name = 'Kreisfreie Städte u. Landkreise',
        header = 5,
        index_col = 0)
    kreise = kreise.rename(columns = {'2': 'Bundesland', 3: 'Kreis', 6: 'Einwohnerzahl'})[['Bundesland', 'Kreis', 'Einwohnerzahl']]
    kreise.index.set_names("Key", inplace = True)
    return kreise

In [ ]:
class ColumnsAdder:

    def __init__(self, kreise):
        self.kreise = kreise

    def addKreisAndBundeslandAndEinwohnerzahlColumns(self, dataFrame):
        dataFrame_kreise = pd.merge(dataFrame, self.kreise, how = 'left', left_on = 'gemeindeschluessel', right_index = True)
        dataFrame['Kreis'] = dataFrame_kreise['Kreis']
        dataFrame['Einwohnerzahl'] = dataFrame_kreise['Einwohnerzahl']
        return self._addBundeslandColumn(dataFrame)
        
    def _addBundeslandColumn(self, dataFrame):
        return pd.merge(
            dataFrame,
            self._createBundeslandByKeyTable(),
            how = 'left',
            left_on = 'bundesland',
            right_index = True)

    def _createBundeslandByKeyTable(self):
        return self.kreise[self.kreise.index.str.len() == 2][['Bundesland']]


In [ ]:
timeSeries = ColumnsAdder(readKreise()).addKreisAndBundeslandAndEinwohnerzahlColumns(readTimeseries())
timeSeries

In [ ]:
kreisValues = timeSeries['Kreis'].drop_duplicates().values
kreisValues

In [ ]:
def printKreisOptions(kreisValues):
    for kreis in kreisValues:
        printKreisOption(kreis)

def printKreisOption(kreis):
    print('<option value="{kreis}">{kreis}</option>'.format(kreis = kreis))

In [ ]:
kreisValues = sorted(kreisValues)
printKreisOptions(kreisValues)

In [ ]:
import os


class IOUtils:

    def saveDictAsJson(dict, file):
        IOUtils.ensurePath(file)
        with open(file, 'w') as outfile:
            json.dump(dict, outfile)

    @staticmethod
    def ensurePath(file):
        directory = os.path.dirname(file)
        if not os.path.exists(directory):
            os.makedirs(directory)


In [ ]:
def getIntensiveCareBeds(timeSeries, kreis = None):
    if kreis is not None:
        return timeSeries[timeSeries['Kreis'] == kreis][['date', 'betten_belegt', 'betten_frei', 'Einwohnerzahl']]
    else:
        return timeSeries.groupby('date').agg(**{
                        'betten_belegt': pd.NamedAgg(column = 'betten_belegt', aggfunc = 'sum'),
                        'betten_frei':   pd.NamedAgg(column = 'betten_frei',   aggfunc = 'sum'),
                        'Einwohnerzahl': pd.NamedAgg(column = 'Einwohnerzahl', aggfunc = 'sum') 
                    }).reset_index()

In [ ]:
import json


def getAndPersistIntensiveCareBeds(timeSeries, kreis=None):
    dataFrame = getIntensiveCareBeds(timeSeries, kreis)
    display(kreis)
    _saveDataFrameAsJson(dataFrame, _getFilename(kreis))
    return dataFrame


def _saveDataFrameAsJson(dataFrame, file):
    IOUtils.saveDictAsJson(
        {
            'population': int(dataFrame.iloc[0]['Einwohnerzahl']),
            'data': _dataFrame2Dict(dataFrame),
        },
        file)


def _dataFrame2Dict(dataFrame):
    df = dataFrame[['date', 'betten_belegt', 'betten_frei']]
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    return df.to_dict(orient="records")


def _getFilename(kreis):
    return '../../docs/data/intensivstationen/intensivstationen-{suffix}.json'.format(suffix=_getSuffix(kreis))


def _getSuffix(kreis):
    return kreis if kreis is not None else 'de'


In [ ]:
getAndPersistIntensiveCareBeds(timeSeries)

In [ ]:
for kreis in kreisValues:
    getAndPersistIntensiveCareBeds(timeSeries, kreis)